In [1]:
import numpy as np
import pandas as pd
from simple.renko import Renko
from simple.jurik import JTPO, JRSX
from simple.plotly import interactFigure
from simple.pretty import pmap
from simple.hurst import HurstRS, HurstDSOD

In [2]:
C = np.load('data/ETHUSDT.2022-05-01.debounce.npz')['ETHUSDT'].view(np.recarray)
len(C), C

(7684832,
 rec.array([('2022-05-01T00:00:06.912000',       0, 2726.665, 6300000, -5.5435e+04,  922., -5.6357e+04, 51, 16, 35),
            ('2022-05-01T00:00:10.695999',       1, 2726.635, 3783999, -3.3200e+03, 7235., -1.0555e+04, 17, 10,  7),
            ('2022-05-01T00:00:10.724999',       2, 2726.565,       0, -2.8000e+01,    0., -2.8000e+01,  1,  0,  1),
            ...,
            ('2022-05-31T23:59:56.573000', 7684829, 1941.785,   20000,  2.3700e+02,  237.,  0.0000e+00,  2,  2,  0),
            ('2022-05-31T23:59:59.116999', 7684830, 1941.825, 2543999, -1.3880e+03, 2137., -3.5250e+03, 11,  8,  3),
            ('2022-05-31T23:59:59.116999', 7684831, 1941.875,       0,  1.0000e+02,  100.,  0.0000e+00,  1,  1,  0)],
           dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')]))

In [3]:
count = 30

In [4]:
def create(step):
    rnk = Renko(step, C['Price'])
    rnk.create_renko()

    X = pd.DataFrame(rnk.bricks)
    X['mid'] = (X.close + X.open) / 2
    return X.mid.values

In [5]:
R = pmap(create, range(1, count))

  0%|          | 0/29 [00:00<?, ?it/s]

In [9]:
def model(step: int = (0, count-1, 1), period: int = (4, 100, 2)):
    renko = R[step]
    TPO = JTPO(renko, period)
    RSX = JRSX(renko, period)
    Hurst_DSOD = HurstDSOD(renko, period)
    Hurst_RS = HurstRS(np.diff(renko), period)
    return 0, locals()

In [11]:
interactFigure(model, rows=3,
    renko={'color': 'gray', 'opacity': 0.4, 'mode': 'markers+lines'}, 
    TPO={'color': 'orange', 'row': 2},
    RSX={'color': 'red', 'row': 2, 'secondary_y': True},
    Hurst_DSOD={'color': 'blue', 'row': 3},
    Hurst_RS={'color': 'gray', 'row': 3, 'secondary_y': True}
)